In [66]:
import sys,  getopt,  time 
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import os

def make_inp(inp):
    #parse inp to needed info
    k=inp.find("kin")
    und=inp[k:].find("_")
    kin=inp[k+3:und]
    und2=inp[und+1:].find("_")
    tgt=inp[und+1:und2+und+1]
    mf=inp[und+1+und2:].find("m")
    model=inp[und+1+und2+mf+1:und+1+und2+mf+2]
    file_n="INP/"+inp
    file=open(file_n,"w")
    file.write("input files for EXTERNAL: runplan/target/output\n")
    file.write("RUNPLAN/kin{}_ep.inp\n".format(kin))
    file.write("TARG/targ.{}_{}\n".format(tgt,model))
    file.write("OUT/kin{}out{}_m.out".format(kin,tgt,model))
    file.close() 
    
def isnewer(file1,file2):
    if not os.path.exists(file1) or not os.path.exists(file2):
        print("No file")
        return False
    
    stats_f1 = os.stat(file1)
    time1=stats_f1[8]
    stats_f2 = os.stat(file2)
    time2=stats_f2[8]
    
    if time1 >= time2:
        return True
    else:
        return False
    
def rerun(inps,tgt="",kin="",m=""):
    for inp in inps:
        if tgt in inp:
            if "kin"+kin in inp:
                if "m"+m in inp:
                    print("Rerunning ",inp)
                    arg=inp[:-4]
                    !sh run_extern $arg
                    


In [ ]:
## WIP

import sys, getopt

def main(argv):
    inputfile = ''
    outputfile = ''
    try:
        opts, args = getopt.getopt(argv,"hi:o:",["ifile=","ofile="])
    except getopt.GetoptError:
        print 'test.py -i <inputfile> -o <outputfile>'
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print 'test.py -i <inputfile> -o <outputfile>'
            sys.exit()
        elif opt in ("-i", "--ifile"):
            inputfile = arg
        elif opt in ("-o", "--ofile"):
            outputfile = arg
    print 'Input file is "', inputfile
    print 'Output file is "', outputfile
    
    
    

if __name__ == "__main__":
     main(sys.argv[1:])

In [87]:
#Build list of possible inps

chk_file = "OUT/kin0outC12.out"
targets=["H2","D2","C12","B10","B11"]
models =[1,3]
kins   =[0,1]

inps = []

for tgt in targets:
    for m in models:
        for kin in kins:
            inps.append("kin%i_%s_m%i.inp"%(kin,tgt,m))


#build list of ones needed to run by coparing the output file with chk_file            
need_to_run=[]            
skipping=[]            
for inp in inps:
    #check if input file is there
    path= "INP/"+inp    
    if not os.path.exists(path):
        make_inp(inp)
        print("makeing ",inp)
        
    #Maybe add target file check
    
    #Compare the output see if needs to be remade
    outfile = "OUT/"+inp.replace("inp","out")
    if  os.path.exists(outfile):
        if isnewer(chk_file,outfile):
            skipping.append(inp)
    else:
        need_to_run.append(inp)
        
    

print()
print(need_to_run)




['kin0_H2_m1.inp', 'kin1_H2_m1.inp', 'kin0_H2_m3.inp', 'kin1_H2_m3.inp', 'kin0_D2_m1.inp', 'kin1_D2_m1.inp', 'kin0_D2_m3.inp', 'kin1_D2_m3.inp', 'kin0_C12_m1.inp', 'kin1_C12_m1.inp', 'kin0_C12_m3.inp', 'kin1_C12_m3.inp', 'kin0_B10_m1.inp', 'kin1_B10_m1.inp', 'kin0_B10_m3.inp', 'kin1_B10_m3.inp', 'kin0_B11_m1.inp', 'kin1_B11_m1.inp', 'kin0_B11_m3.inp', 'kin1_B11_m3.inp']
['kin1_B10_m1.inp', 'kin0_B10_m3.inp', 'kin1_B10_m3.inp', 'kin0_B11_m1.inp', 'kin1_B11_m1.inp', 'kin0_B11_m3.inp', 'kin1_B11_m3.inp']


In [83]:
skipping

['kin1_D2_m1.inp', 'kin1_D2_m3.inp', 'kin0_C12_m1.inp', 'kin1_C12_m1.inp']

In [88]:
# Run the the need_to_run list

for inp in need_to_run:
    arg=inp[:-4]
    print("Runing ", arg)
    !sh run_extern $arg

Runing  kin1_B10_m1
Runing  kin0_B10_m3
Runing  kin1_B10_m3
Runing  kin0_B11_m1
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Runing  kin1_B11_m1
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Runing  kin0_B11_m3
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG
Runing  kin1_B11_m3
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG


In [100]:
rerun(inps,"H2","1","3")

Rerunning  kin1_H2_m3.inp


In [ ]:
erw